# Pulling ETF data together

In [71]:
import pandas as pd
from statistics import median

import warnings
warnings.filterwarnings('ignore')

In [2]:
# lazy load etf forecasts and etfs
forecasts = context.catalog.load('etf_forecasts')
etfs = context.catalog.load('etf_historical')

2021-02-07 20:15:54,297 - kedro.io.data_catalog - INFO - Loading data from `etf_forecasts` (PartitionedDataSet)...
2021-02-07 20:15:54,780 - kedro.io.data_catalog - INFO - Loading data from `etf_historical` (PartitionedDataSet)...


In [4]:
# load specific random etf, which we know data exists for
fcast = forecasts['etf_VUKG_IE00BFMXYP42']()

In [13]:
# filter on todays date
fcast[fcast.ds == pd.to_datetime('today').strftime("%Y-%m-%d")]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
405,2021-02-03,30.296369,29.193687,31.288277,30.296369,30.307866,-0.039503,-0.039503,-0.039503,-0.039503,-0.039503,-0.039503,0.0,0.0,0.0,30.256866


In [6]:
# extract typical historical etf data
etf = etfs['etf_VUKG_IE00BFMXYP42']()
etf.head(20)

,Date,Open,High,Low,Close,Volume,Currency,Exchange
0,2019-05-22,30.75,30.75,30.75,30.59,20000,GBP,London
1,2019-05-23,30.12,30.12,30.12,30.12,600,GBP,London
2,2019-06-02,29.89,29.89,29.89,29.89,0,GBP,London
3,2019-06-05,30.18,30.18,30.18,30.09,20000,GBP,London
4,2019-06-07,30.57,30.57,30.57,30.57,0,GBP,London
5,2019-06-15,30.66,30.66,30.66,30.66,0,GBP,London
6,2019-06-16,30.66,30.66,30.66,30.66,0,GBP,London
7,2019-06-18,30.88,30.88,30.88,31.14,20000,GBP,London
8,2019-06-19,30.99,30.99,30.99,30.88,1670,GBP,London
9,2019-06-20,31.16,31.16,31.02,31.04,454,GBP,London


In [41]:
# convert Date column to date format
etf['Date'] = pd.to_datetime(etf.Date)

In [44]:
# get min and max of dates
mn, mx = min(etf.Date), max(etf.Date)

In [45]:
# calculate age of data (in years)
(mx - mn).days / 365

1.6904109589041096

In [48]:
# create year column
etf['Year'] = etf.Date.dt.year
etf.Year

0      2019
1      2019
2      2019
3      2019
4      2019
       ... 
395    2021
396    2021
397    2021
398    2021
399    2021
Name: Year, Length: 400, dtype: int64

In [49]:
# group data by years
grouped = etf.groupby('Year')

In [61]:
# extract min and max per year
mn, mx = grouped.Close.min(), grouped.Close.max()

In [73]:
# calculate annual median rate of return
median((mx - mn) / mn)

0.09781512605042006